### Começo

In [18]:
import json
from tqdm import tqdm

In [2]:
PATH  = "C:/Users/rbdf/Documents/Datasets/PraCegoVer/images/"
file = "C:\\Users\\rbdf\\Documents\\Datasets\\PraCegoVer\\dataset.json"

In [5]:
# reading original json
with open(file, 'r') as f:
    annotations = json.load(f)

In [11]:
annotations[0]['filename']

'i-00000000.jpg'

In [22]:
# First attempt to find corrupted files
from struct import unpack
import os


marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]            
            if len(data)==0:
                break        

bads = []

for img in tqdm(annotations):
    img_file = PATH+img['filename']
    # print(PATH+img['filename'])
    # image = osp.join(root_img,img)
    image = JPEG(img_file) 
    try:
        image.decode()   
    except:
        bads.append(img_file)
    # break
    

100%|██████████| 533523/533523 [20:46<00:00, 427.97it/s] 


In [23]:
print(len(annotations))

for k in bads:
    del annotations[int(k[-12:-4])]
print(len(annotations))

533523
533510


In [ ]:
import json
with open('Annotations_PraCegoVer_fix.json', 'w') as f:
    json.dump(annotations, f)